<a href="https://colab.research.google.com/github/NekoDK/Omgu_DA/blob/OmguDA_3/OmguDA_3%2C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


In [3]:
df = pd.read_excel("SrEd.xlsx")

In [4]:
edu_dummies = pd.get_dummies(df['Edu'], prefix='is')
df = pd.concat([df, edu_dummies], axis=1)
df.drop('Edu', axis=1, inplace=True)
df.drop('id', axis=1, inplace=True)

In [5]:
df_female = df[df['Sex'] != 'м'].copy()
df_male = df[df['Sex'] == 'м'].copy()

df_female.drop('Sex', axis=1, inplace=True)
df_male.drop('Sex', axis=1, inplace=True)

In [7]:
# Обучение модели для мужчин
X_male = df_male.drop('Srok', axis=1)
y_male = df_male['Srok'].values

model_male = LinearRegression()
model_male.fit(X_male, y_male)

y_pred_male = model_male.predict(X_male)
mse_male = mean_squared_error(y_male, y_pred_male)
print("Среднеквадратическая ошибка (М):", mse_male)

Среднеквадратическая ошибка (М): 7.8832004334692005


In [8]:
# Обучение модели для женщин
X_female = df_female.drop('Srok', axis=1)
y_female = df_female['Srok'].values

model_female = LinearRegression()
model_female.fit(X_female, y_female)

y_pred_female = model_female.predict(X_female)
mse_female = mean_squared_error(y_female, y_pred_female)
print("Среднеквадратическая ошибка (Ж):", mse_female)

Среднеквадратическая ошибка (Ж): 1.3803392180038194


In [9]:
# Обучение модели для всех данных
df.drop('Sex', axis=1, inplace=True)

X_all = df.drop('Srok', axis=1)
y_all = df['Srok'].values

model_all = LinearRegression()
model_all.fit(X_all, y_all)

y_pred_all = model_all.predict(X_all)
mse_all = mean_squared_error(y_all, y_pred_all)
print("Среднеквадратическая ошибка (Общая):", mse_all)


Среднеквадратическая ошибка (Общая): 6.3673835986977965


In [10]:
def sum_of_squares(y_true, y_pred):
    return np.sum((y_true - y_pred) ** 2)

In [11]:
# Вычисление RSS
rss_male = sum_of_squares(y_male, y_pred_male)
rss_female = sum_of_squares(y_female, y_pred_female)
rss_all = sum_of_squares(y_all, y_pred_all)

k = X_male.shape[1]
n_male = X_male.shape[0]
n_female = X_female.shape[0]
n_all = X_all.shape[0]

In [12]:
# Критерий Чоу
f_stat = ((rss_all - (rss_male + rss_female)) / k) / ((rss_male + rss_female) / (n_male + n_female - 2 * k))
print("Критерий Чоу:", f_stat)

Критерий Чоу: 0.4024602020577145


In [13]:
# Критическое значение F-распределения
from scipy.stats import f

alpha = 0.05
df1 = k
df2 = n_male + n_female - 2 * k

f_critical = f.ppf(1 - alpha, df1, df2)
print(f"Critical F-value: {f_critical}")

Critical F-value: 2.4494664263887103


In [14]:
if f_stat > f_critical:
    print(" Существует структурный разрыв.")
else:
    print(" Структурного разрыва нет.")

 Структурного разрыва нет.
